# Production Pydantic Models for Gen AI

Base models and patterns for LLM applications, APIs, and data validation.

## Setup

In [ ]:
# !pip install pydantic

## 1. Base Response Models

In [ ]:
"""Standard API response models."""

from pydantic import BaseModel, Field
from typing import Any, Generic, Literal, TypeVar
from datetime import datetime

T = TypeVar("T")


class APIResponse(BaseModel, Generic[T]):
    """Generic API response wrapper."""
    success: bool
    data: T | None = None
    error: str | None = None
    error_code: str | None = None
    timestamp: datetime = Field(default_factory=datetime.utcnow)

    @classmethod
    def ok(cls, data: T) -> "APIResponse[T]":
        return cls(success=True, data=data)
    
    @classmethod 
    def fail(cls, error: str, error_code: str = "ERROR") -> "APIResponse[T]":
        return cls(success=False, error=error, error_code=error_code)


class PaginatedResponse(BaseModel, Generic[T]):
    """Paginated list response."""
    items: list[T]
    total: int
    page: int
    page_size: int
    has_next: bool
    has_prev: bool


# Test
class User(BaseModel):
    id: int
    name: str

response = APIResponse[User].ok(User(id=1, name="Alice"))
print(response.model_dump_json(indent=2))

## 2. LLM Message Models

In [ ]:
"""Models for LLM chat interactions."""

from pydantic import BaseModel, Field
from typing import Literal


class Message(BaseModel):
    """Single chat message."""
    role: Literal["system", "user", "assistant"]
    content: str


class ChatRequest(BaseModel):
    """Request to LLM API."""
    messages: list[Message]
    model: str = "gpt-4"
    temperature: float = Field(default=0.7, ge=0, le=2)
    max_tokens: int | None = None
    stream: bool = False


class Usage(BaseModel):
    """Token usage stats."""
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int


class ChatResponse(BaseModel):
    """Response from LLM API."""
    content: str
    model: str
    usage: Usage
    finish_reason: Literal["stop", "length", "content_filter"] | None = None


# Test
request = ChatRequest(
    messages=[
        Message(role="system", content="You are a helpful assistant."),
        Message(role="user", content="Hello!"),
    ],
    temperature=0.5
)
print(request.model_dump_json(indent=2))

## 3. Structured Output Models

In [ ]:
"""Models for structured LLM output parsing."""

from pydantic import BaseModel, Field
from typing import Literal


class ExtractedEntity(BaseModel):
    """Entity extracted from text."""
    name: str
    entity_type: str
    confidence: float = Field(ge=0, le=1)
    start_char: int | None = None
    end_char: int | None = None


class Sentiment(BaseModel):
    """Sentiment analysis result."""
    label: Literal["positive", "negative", "neutral"]
    score: float = Field(ge=0, le=1)
    explanation: str | None = None


class Classification(BaseModel):
    """Text classification result."""
    category: str
    confidence: float = Field(ge=0, le=1)
    subcategories: list[str] = []


class Summary(BaseModel):
    """Text summarization result."""
    summary: str
    key_points: list[str]
    word_count: int


class QAResult(BaseModel):
    """Question answering result."""
    answer: str
    confidence: float = Field(ge=0, le=1)
    sources: list[str] = []
    is_answerable: bool = True


# Test
result = Sentiment(label="positive", score=0.92, explanation="Enthusiastic tone")
print(result.model_dump_json())

## 4. RAG Models

In [ ]:
"""Models for RAG (Retrieval Augmented Generation) pipelines."""

from pydantic import BaseModel, Field
from typing import Any


class Document(BaseModel):
    """Document chunk for RAG."""
    id: str
    content: str
    metadata: dict[str, Any] = {}
    embedding: list[float] | None = None


class SearchResult(BaseModel):
    """Search result from vector store."""
    document: Document
    score: float
    rank: int


class RAGQuery(BaseModel):
    """RAG query request."""
    query: str
    k: int = Field(default=5, ge=1, le=20)
    filter: dict[str, Any] | None = None
    rerank: bool = False


class RAGResponse(BaseModel):
    """RAG query response."""
    answer: str
    sources: list[SearchResult]
    query: str
    model: str


# Test
doc = Document(
    id="doc_001",
    content="Python is a programming language.",
    metadata={"source": "wiki", "page": 1}
)
print(doc.model_dump_json(indent=2))

## 5. Configuration Models

In [ ]:
"""Application configuration models."""

from pydantic import BaseModel, Field, SecretStr
from typing import Literal


class LLMConfig(BaseModel):
    """LLM provider configuration."""
    provider: Literal["openai", "anthropic", "google"]
    model: str
    api_key: SecretStr
    temperature: float = Field(default=0.7, ge=0, le=2)
    max_tokens: int = 4096
    timeout: float = 30.0


class VectorStoreConfig(BaseModel):
    """Vector store configuration."""
    provider: Literal["pinecone", "weaviate", "chroma", "qdrant"]
    index_name: str
    dimension: int = 1536
    metric: Literal["cosine", "euclidean", "dot"] = "cosine"


class AppConfig(BaseModel):
    """Main application configuration."""
    app_name: str
    debug: bool = False
    log_level: Literal["DEBUG", "INFO", "WARNING", "ERROR"] = "INFO"
    llm: LLMConfig
    vector_store: VectorStoreConfig | None = None


# Test
config = AppConfig(
    app_name="MyApp",
    llm=LLMConfig(
        provider="openai",
        model="gpt-4",
        api_key="sk-secret"
    )
)
print(config.model_dump_json(indent=2))

## All-in-One: models.py

In [ ]:
# models.py - Copy to create module
"""
Pydantic models for Gen AI applications.

Usage:
    from models import ChatRequest, Message, Document, RAGQuery
"""

__all__ = [
    # API
    "APIResponse",
    "PaginatedResponse",
    # LLM
    "Message",
    "ChatRequest",
    "ChatResponse",
    "Usage",
    # Structured Output
    "ExtractedEntity",
    "Sentiment",
    "Classification",
    "Summary",
    "QAResult",
    # RAG
    "Document",
    "SearchResult",
    "RAGQuery",
    "RAGResponse",
    # Config
    "LLMConfig",
    "VectorStoreConfig",
    "AppConfig",
]